# Continuous Control

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [ ]:
from unityagents import UnityEnvironment

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Reacher.app"`
- **Windows** (x86): `"path/to/Reacher_Windows_x86/Reacher.exe"`
- **Windows** (x86_64): `"path/to/Reacher_Windows_x86_64/Reacher.exe"`
- **Linux** (x86): `"path/to/Reacher_Linux/Reacher.x86"`
- **Linux** (x86_64): `"path/to/Reacher_Linux/Reacher.x86_64"`
- **Linux** (x86, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86"`
- **Linux** (x86_64, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86_64"`

For instance, if you are using a Mac, then you downloaded `Reacher.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Reacher.app")
```

In [ ]:
env = UnityEnvironment(file_name='Reacher.app')

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Train an agent

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [ ]:
import torch    
import torch.autograd as autograd         
import torch.nn as nn                     
import torch.nn.functional as F           
import torch.optim as optim               
import torch.distributions

import pandas as pd
import numpy as np

import holoviews as hv
from holoviews import opts
from holoviews.streams import Pipe, Buffer

import streamz
import streamz.dataframe

import pdb, gym

hv.extension('bokeh')

In [ ]:
# Setup training progress output
def init_training_progress():

    max_length = 1000000
    rolling_size = 25

    training_stream = streamz.Stream()
    example = pd.DataFrame({'x': [0]}, index=[0])

    training_sdf = streamz.dataframe.DataFrame(training_stream, example=example)

    training_raw_buffer = Buffer(training_sdf, length=max_length)
    training_smooth_buffer = Buffer(training_sdf.x.rolling(rolling_size).median())

    training_raw_dmap = hv.DynamicMap(hv.Curve, streams=[training_raw_buffer]).relabel('raw')
    training_smooth_dmap = hv.DynamicMap(hv.Curve, streams=[training_smooth_buffer]).relabel('smooth')
    
    return training_stream, training_raw_dmap, training_smooth_dmap

In [ ]:
class PolicyNet(nn.Module):
    
    def __init__(self, state_size, action_size):
        super(PolicyNet, self).__init__()
        
        # Hidden layers
        hidden_size = 32
        self.fc1 = nn.Linear(state_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        
        # Output layer of action means
        self.fc3 = nn.Linear(hidden_size, action_size)
        
        # Standard deviations approximated seperately
        self.register_parameter('log_sigma', None)
        self.log_sigma = nn.Parameter(torch.zeros(action_size), requires_grad=True)        

    def forward(self, x):
        
        x = torch.tanh(self.fc1(x))
        x = torch.tanh(self.fc2(x))
        
        means = torch.tanh(self.fc3(x))
        sigmas = torch.exp(self.log_sigma).expand(means.shape)
        
        return means, sigmas

In [ ]:
class ValueNet(nn.Module):
    
    def __init__(self, state_size):
        super(ValueNet, self).__init__()
        
        # Hidden layers
        hidden_size = 32
        self.fc1 = nn.Linear(state_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        
        # Output layer - single state value
        self.fc3= nn.Linear(hidden_size, 1)

    def forward(self, x):
        
        x = torch.tanh(self.fc1(x))
        x = torch.tanh(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        
        return x

In [ ]:
class Agent():
    
    def __init__(self, state_size, action_size, lr=1e-3):
        self.lr = lr
        self.actor = PolicyNet(state_size, action_size)
        self.critic = ValueNet(state_size)
        self.actor_optimiser = optim.Adam(self.actor.parameters(), lr=self.lr)
        self.critic_optimiser = optim.Adam(self.critic.parameters(), lr=self.lr)

    def start_episode(self):
        self.episode_rewards = []

    def act(self, state):
        
        # Derive action distribution from the policy network (actor)
        means, sigmas = self.actor(state)
        action_distribution = torch.distributions.Normal(means, sigmas)
        action = action_distribution.sample()
        action_log_prob = action_distribution.log_prob(action)
        
        return state, action, action_log_prob
    
    def record_outcome(self, state, action, action_log_prob, reward, next_state):
        
        # Record reward for assesment
        self.episode_rewards.append(reward)
        
        # Use the critic to estimate state values
        state_value = self.critic(state).squeeze()
        next_state_value = self.critic(next_state).squeeze()
        
        # Calculate the TD target/error and detach from the computation graph
        td_target = (reward + next_state_value).detach()
        td_error  = (reward + next_state_value - state_value).detach()
        
        # Train the agent
        self.train_critic(td_target, state_value)
        self.train_actor(td_error, action_log_prob)

    def train_critic(self, td_target, state_value):
        
        loss = F.mse_loss(td_target, state_value)
        
        self.critic_optimiser
        loss.backward()
        self.critic_optimiser.step()
    
    def train_actor(self, td_error, action_log_prob):
        
        loss = -(td_error * action_log_prob.sum(dim=1)).mean()
        
        self.actor_optimiser.zero_grad()
        loss.backward()
        self.actor_optimiser.step()

    def average_episode_return(self):
        return sum([r.mean().item() for r in self.episode_rewards])


In [ ]:
def play_episode_and_learn(env, agent, max_episode_length=1000):
    
    # Run concurrent episode on all environments
    env_info = env.reset(train_mode=True)[brain_name]
    state = torch.from_numpy(env_info.vector_observations).float()
    
    # Start episode
    agent.start_episode()
    
    for _ in range(max_episode_length):    
        
        # Calculate actions for all envs
        state, action, action_log_prob = agent.act(state)
        
        # Run through the envs in parallel
        env_info = env.step(action.numpy())[brain_name]
        next_state = torch.from_numpy(env_info.vector_observations).float()
        reward = torch.tensor(env_info.rewards).float()
        done = env_info.local_done
        
        # Record the experience tuple with the agent
        agent.record_outcome(state, action, action_log_prob, reward, next_state)
        
        # Advance
        state = next_state

        # We want rectangular input to network, so if any finish early we finish all early
        if np.any(done):
            print("Someone finished")
            break
    


In [ ]:
# Create policy and agent
state_size = brain.vector_observation_space_size
action_size = brain.vector_action_space_size
agent = Agent(state_size, action_size, lr=1e-3)
episode_index = 0

In [ ]:
# Display training progress
stream, smooth, raw = init_training_progress()
layout = (smooth * raw)
layout.opts(
    opts.Curve(width=900, height=300, show_grid=True, tools=['hover'])
)

In [ ]:
# Train
num_episodes = 5
max_episode_length = 1000

for _ in range(num_episodes):
    
    play_episode_and_learn(env, agent, max_episode_length=max_episode_length)
    
    average_episode_return = (agent.average_episode_return() / max_episode_length) * 1000
    stream.emit( pd.DataFrame({'x': average_episode_return}, index=[episode_index]) )
    episode_index += 1


In [ ]:
env.close()

In [ ]:
# Show params
for x in agent.subagents[0].policy.parameters():
    print(x)


In [ ]:
# Run agent in the environment
env_info = env.reset(train_mode=False)[brain_name]      # reset the environment
states = torch.from_numpy(env_info.vector_observations).float()
num_agents = len(env_info.agents)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
for i in range(100):
    actions = agent.act(states)
    env_info = env.step(actions.numpy())[brain_name]
    rewards = env_info.rewards                         # get reward (for each agent)
    states = torch.from_numpy(env_info.vector_observations).float()
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    if np.any(dones):                                  # exit loop if episode finished
        break

print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

In [ ]:

env_info = env.reset(train_mode=True)[brain_name] 
env_info.vector_observations.shape


In [ ]:
# Save to disk
torch.save(agent.policy, 'agent.policy')

In [ ]:
# Load from disk
agent.policy = torch.load('agent.policy')
agent.policy.eval()

In [ ]:
torch.manual_seed(0)

state_size = brain.vector_observation_space_size
action_size = brain.vector_action_space_size
agent = Agent(state_size, action_size, lr=3e-4, rollout_length=3)

agent.start_episode()

# Dummy interact with env
states = np.zeros((20,33))

for i in range(3):

    # Get actions
    pdb.set_trace()
    actions = agent.act(torch.from_numpy(states).float())
    
    # Dummy interact with env
    next_states = (i+1) * 0.1 * np.ones((20,33))
    rewards = 0.1 * np.ones(20)
    dones = [False] * 20

    # Teach agent
    agent.record_outcome(rewards)
    
    # Advance
    states = next_states

agent.finish_episode()

In [ ]:
torch.manual_seed(0)
agent = Agent(state_size, action_size, lr=3e-4, rollout_length=3)
for x in agent.policy.parameters():
    print(x)

In [ ]:
env.close()